In [6]:
import base64
import requests
import os


In [7]:
# OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")


In [8]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "/Users/serkanhaytac/experimental-projects/autogen-habittracker/app_screenshot.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What’s in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}


In [9]:

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())


{'id': 'chatcmpl-8MH6yWXpqsoX7jyJQmWomqKwnpwCR', 'object': 'chat.completion', 'created': 1700319684, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 1118, 'completion_tokens': 300, 'total_tokens': 1418}, 'choices': [{'message': {'role': 'assistant', 'content': 'This image shows a screenshot of a habit tracking or health app interface displaying a user\'s daily goals and progress. At the top of the screen, there is a weekly calendar with days from Wednesday to Tuesday, showing that it is currently Tuesday. Below the calendar, there are several health-related activities listed, each with a progress bar or a completed indicator:\n\n1. Drink water - 2450/3000 ml (with a flame icon indicating a 219-day streak)\n2. Yoga - 25min/30min (with the same flame icon for the streak)\n3. Drink Less Beverage - Zero out of one allowed drink (indicated by a flame icon for a 220-day streak)\n4. Eat Breakfast - Completed one out of one (with a flame icon for the streak)\n5. Walk - Complet